In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import json
import numpy as np
import pandas as pd
# import nltk
# nltk.download('punkt')
import time
import math
import string
from torch.utils.data import Dataset, DataLoader
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from typing import Iterable, List
from torch import Tensor
from torch.nn import Transformer
import re
import csv
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy
nlp = spacy.load("en_core_web_sm")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.tokenize import indic_tokenize
import indicnlp

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1399 (delta 135), reused 147 (delta 120), pack-reused 1219
Receiving objects: 100% (1399/1399), 9.57 MiB | 11.46 MiB/s, done.
Resolving deltas: 100% (745/745), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 19.66 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [ ]:
MAX_LENGTH=20
LANGUAGES = ['Kannada','Hindi','Bengali','Gujarati','Telgu','Malayalam','Tamil',]
z = 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    translator = str.maketrans('', '', string.punctuation + "\"'")
    s = s.translate(translator)
    return s.strip()

def extract_data(language,filepath):
  with open(filepath, 'r') as file:
    data=json.load(file)
  source_sentences = []
  target_sentences = []
  id = []
  for language_pair, language_data in data.items():
      print(f"Language Pair: {language_pair}")
      if(language_pair==f'English-{language}'):
        for data_type, data_entries in language_data.items():
            print(f"  Data Type: {data_type}")
            for entry_id, entry_data in data_entries.items():
              source_sentences.append(normalizeString(entry_data["source"]))
              if "target" in entry_data:
                target_sentences.append(normalizeString(entry_data["target"]))
              id.append(entry_id)
  return source_sentences,target_sentences,id

In [ ]:
source_sentence,target_sentence,id = extract_data(LANGUAGES[z] , '/content/drive/MyDrive/drive/train_data2.json')

Language Pair: English-Bengali
Language Pair: English-Gujarati
Language Pair: English-Hindi
Language Pair: English-Kannada
  Data Type: Train
Language Pair: English-Malayalam
Language Pair: English-Tamil
Language Pair: English-Telgu


In [ ]:
source_sentence[:1],target_sentence[:1],id[:1],len(source_sentence),len(target_sentence)

(['ಅವರು ಒಂದೇ ಮಾರುಕಟ್ಟೆಯ ಸ್ಥಳದಲ್ಲಿ ಸಮೀಪದ ಐದು ಮಾರುಕಟ್ಟೆಗಳ ಇರುವಿಕೆಯನ್ನು ಒಪ್ಪಿಕೊಂಡಿದ್ದಾರೆ'],
 ['at the place of lone market he has considered the presence of five nearby markets'],
 ['563224'],
 46794,
 46794)

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, list1, list2):
        self.list1 = list1
        self.list2 = list2

    def __len__(self):
        return len(self.list1)

    def __getitem__(self, idx):
        concatenated_item = (self.list1[idx], self.list2[idx])
        return concatenated_item

SRC_LANGUAGE = LANGUAGES[z]
TGT_LANGUAGE = 'English'

# Place-holders
token_transform = {}
vocab_transform = {}
class tokens():
  def __init__(self,lang):
    self.lang=lang
  def __call__(self, data):
    return indic_tokenize.trivial_tokenize(data)

In [ ]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy')
# token_transform[TGT_LANGUAGE] = get_tokenizer('spacy')
token_transform[TGT_LANGUAGE] = tokens(LANGUAGES[z])
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample)


UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
train_iter_src,train_iter_tgt=[],[]
for t in range(7):
  iter_src,iter_tgt,id =extract_data(LANGUAGES[t] , '/content/drive/MyDrive/drive/train_data2.json')
  train_iter_src.extend(iter_src)
  train_iter_tgt.extend(iter_tgt)
# Create torchtext's Vocab object
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_iter_src,SRC_LANGUAGE ),
                                                min_freq=2,
                                                specials=special_symbols,
                                                special_first=True)
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_iter_tgt, TGT_LANGUAGE),
                                                min_freq=2,
                                                specials=special_symbols,
                                                special_first=True)
# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)


/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


Language Pair: English-Bengali
Language Pair: English-Gujarati
Language Pair: English-Hindi
Language Pair: English-Kannada
  Data Type: Train
Language Pair: English-Malayalam
Language Pair: English-Tamil
Language Pair: English-Telgu
Language Pair: English-Bengali
Language Pair: English-Gujarati
Language Pair: English-Hindi
  Data Type: Train
Language Pair: English-Kannada
Language Pair: English-Malayalam
Language Pair: English-Tamil
Language Pair: English-Telgu
Language Pair: English-Bengali
  Data Type: Train
Language Pair: English-Gujarati
Language Pair: English-Hindi
Language Pair: English-Kannada
Language Pair: English-Malayalam
Language Pair: English-Tamil
Language Pair: English-Telgu
Language Pair: English-Bengali
Language Pair: English-Gujarati
  Data Type: Train
Language Pair: English-Hindi
Language Pair: English-Kannada
Language Pair: English-Malayalam
Language Pair: English-Tamil
Language Pair: English-Telgu
Language Pair: English-Bengali
Language Pair: English-Gujarati
Langu

In [ ]:
len(vocab_transform[SRC_LANGUAGE]),len(vocab_transform[TGT_LANGUAGE])

(264511, 75732)

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE =128
NHEAD = 4
FFN_HID_DIM = 128
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
from torch.utils.data import DataLoader
# train_iter_src,train_iter_tgt,id = source_sentence,target_sentence,id = extract_data(LANGUAGES[z] , '/content/drive/MyDrive/drive/train_data2.json')
print(len(train_iter_src))
def final_data(train_src,train_tgt):
    train_x=[]
    train_y=[]
    for x,y in zip(train_src,train_tgt):
      x=text_transform[SRC_LANGUAGE](x.rstrip("\n"))
      y=text_transform[TGT_LANGUAGE](y.rstrip("\n"))
      len_x=len(x)
      len_y=len(y)
      x1=x
      y1=y
      zeros_x = np.count_nonzero(x1.numpy() == 0.0)
      zeros_y = np.count_nonzero(y1.numpy() == 0.0)
      if zeros_x/len_x<0.2 and zeros_y/len_y<0.2:
        train_x.append(x)
        train_y.append(y)
    return train_x,train_y
train_iter_src,train_iter_tgt=final_data(train_iter_src,train_iter_tgt)
print(len(train_iter_src))
train_iter = CustomDataset(train_iter_src,train_iter_tgt )
def train_epoch(model, optimizer):
    model.train()
    losses = 0

    # train_iter = TensorDataset(train_iter_src, train_iter_tgt)

    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for src, tgt in train_dataloader:
      src = src.to(DEVICE)
      tgt = tgt.to(DEVICE)

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:, :]
      x=tgt_out.type(torch.LongTensor)
      x=x.to(DEVICE)
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), x.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()

    return losses / len(list(train_dataloader))

401243
355348


In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 0


for epoch in range(13, 18+1):
    start_time = timer()
    if epoch!=1:
      model_path = f'/content/drive/MyDrive/your_model{epoch-1}.pth'
      transformer.load_state_dict(torch.load(model_path))
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    model_path = f'/content/drive/MyDrive/your_model{epoch}.pth'
    torch.save(transformer.state_dict(), model_path)
    # val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {0:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))




/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 13, Train loss: 5.048, Val loss: 0.000, Epoch time = 1827.326s
Epoch: 14, Train loss: 4.985, Val loss: 0.000, Epoch time = 1832.816s
Epoch: 15, Train loss: 4.945, Val loss: 0.000, Epoch time = 1822.759s
Epoch: 16, Train loss: 5.038, Val loss: 0.000, Epoch time = 1816.461s


KeyboardInterrupt: ignored

In [ ]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
# print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

In [ ]:
def results(Transformer,lang):
  source_sentence,_,ids = extract_data(lang , '/content/drive/MyDrive/drive/Test_data2_final.json')
  with open(f'{lang}1.csv', mode='w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write data to the CSV file
    csv_writer.writerow(['Id', 'Translation'])

    for sentence,id in zip(source_sentence,ids):
      words = translate(transformer,sentence)
      csv_writer.writerow([id,words])

In [ ]:
model_path = f'/content/drive/MyDrive/your_model16.pth'
transformer.load_state_dict(torch.load(model_path))
for t in range(5,7):
  results(transformer,LANGUAGES[t])

Language Pair: English-Bengali
Language Pair: English-Gujarati
Language Pair: English-Hindi
Language Pair: English-Kannada
Language Pair: English-Malayalam
  Data Type: Test
Language Pair: English-Tamil
Language Pair: English-Telgu
Language Pair: English-Bengali
Language Pair: English-Gujarati
Language Pair: English-Hindi
Language Pair: English-Kannada
Language Pair: English-Malayalam
Language Pair: English-Tamil
  Data Type: Test
Language Pair: English-Telgu
